In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import cv2

img = cv2.imread(r'Test1.jpg')[:,:,::-1]
plt.imshow(img)
plt.show()

In [ ]:
############
## Resize ##
############
image = cv2.resize(img, (1024, 1024))
plt.imshow(image)
plt.show()

In [ ]:
############
## Split  ##
############

# split the image qually into its four quadrants
width, height, num_channels = img.shape
img_q1 = img[:width//2,:height//2]
img_q2 = img[:width//2, height//2:]
img_q3 = img[width//2:, :height//2]
img_q4= img[width//2:, height//2:]
 
plt.subplots(2, 2, sharex=True, sharey=True)
plt.subplot(2, 2, 1)
plt.imshow(img_q1)
plt.subplot(2, 2, 2)
plt.imshow(img_q2)
plt.subplot(2, 2, 3)
plt.imshow(img_q3)
plt.subplot(2, 2, 4)
plt.imshow(img_q4)
plt.tight_layout()
plt.show()

In [ ]:
##################
## Ecualization ##
##################

# We split the image into its color channels and then equalize each separately

# Define channel names
channel_names = ['Reds', 'Greens', 'Blues']

# Create subplots with 3 rows and 2 columns
fig, axes = plt.subplots(4, 2, figsize=(10, 15))

# List to store equalized images
eq = []

for i in range(3):
    # Original Channel
    ax_orig = axes[i, 0]
    ax_orig.imshow(img[:, :, i], cmap=channel_names[i])
    ax_orig.set_title('Original')
    ax_orig.axis('off')  # Hide axis ticks
    
    # Equalized Channel
    eq_img = cv2.equalizeHist(img[:, :, i])
    eq.append(eq_img)
    ax_eq = axes[i, 1]
    ax_eq.imshow(eq_img, cmap=channel_names[i])
    ax_eq.set_title('Equalized')
    ax_eq.axis('off')  # Hide axis ticks
    
    # Add text annotation to the left of each row
    fig.text(0.45, 0.9 -i*0.2, f'{channel_names[i]} Channel', fontsize=12, va='center')
# Composition of the three channels
# Original Total
ax_orig = axes[3, 0]
ax_orig.imshow(img)
ax_orig.set_title('Original')
ax_orig.axis('off')  # Hide axis ticks
# Equalized Total
ax_eq = axes[3, 1]
ax_eq.imshow(np.array(eq).transpose(1,2,0))
ax_eq.set_title('Equalized')
ax_eq.axis('off')  # Hide axis ticks
# Add text annotation to the left of each row
fig.text(0.45, 0.3, f'Total image', fontsize=12, va='center')
plt.show()

In [ ]:
###############################
## Binary Image Manipulation ##
###############################

# Given that:
# - We need to count cells that in many cases touch each other
# - We have been told to apply a morphological opening operation

# Load the image and mask.
mask = cv2.imread(r'MASK.tif', cv2.IMREAD_GRAYSCALE)
plt.imshow(mask,cmap='gray')

In [ ]:
# Apply a morphological opening operation to then remove noise from the boundaries
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(mask,cv2.MORPH_OPEN,kernel, iterations = 3)
plt.imshow(opening,cmap='gray')

In [ ]:
# Finding sure background area
sure_bg = cv2.dilate(opening,kernel,iterations=1)
plt.imshow(sure_bg,cmap='gray')

In [ ]:
# Finding sure foreground area
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,5)
ret, sure_fg = cv2.threshold(dist_transform,0.5*dist_transform.max(),255,0)
plt.imshow(sure_fg,cmap='gray')

In [ ]:
# Marker labelling
ret, markers = cv2.connectedComponents(sure_fg)
# Finally, the maximum label in markers correponds to the number of cells
num_cells = markers.max()
print(f"number of cells: {num_cells}") # this agrees with our count by hand

# The total area (in pixel units) will be the total sum of pixels in the mask
total_area = np.sum(mask)

print(f"average area per cell: {total_area/num_cells:.3f} (pixels)") 

In [ ]:
# Create a copy of the original image for the tinted overlay
tinted_image = img.copy()

# Define the color (blue in BGR) and an alpha for the tint intensity
blue_color = np.array([0, 0, 255], dtype=np.uint8)  
alpha = 0.5  # blending factor for how strong the tint will be

tinted_image[mask>0] = (
    alpha * tinted_image[mask>0] + (1 - alpha) * blue_color
).astype(np.uint8)

plt.imshow(tinted_image)
plt.show()